## Load Data

In [1]:
from opencc import OpenCC
import re
import pandas
import emoji

In [2]:
from pymongo import MongoClient
client = MongoClient()

In [3]:
db = client.test
trData = db.sampleset

In [4]:
import pandas as pd

In [5]:
trData = trData.find({})
trData = list(trData)
df_trData = pd.DataFrame(trData)
df_trData.head()

,_id,author,content,copyrightSource,id,last_update,pubSource,pubTime,title,url
0,5a826ea13ed17ebe58f25c61,丘份碗66,【昌河汽车将推2款新车】与北汽集团整车事业本部副本部长、江西昌河汽车有限责任公司副总经理廖雄...,丘份碗66,-9223352888553609071,2018-01-29 22:48:53,新浪微博,1517050815,丘份碗66,http://weibo.com/6142925293/G0mQ3azgA
1,5a826ea13ed17ebe58f25c62,asd紫衣,这天我差点出事哦，我去买烧烤，公交车，我看到个人像鬼，多少路我忘记了，后来跟着个杨洋的什么广...,asd紫衣,-9223363810167202147,2018-01-15 09:29:57,新浪微博,1515967706,asd紫衣,http://weibo.com/6276804601/FEq78giV6
2,5a826ea13ed17ebe58f25c63,故事与酒缸,【广菲克产销分离JEEP国产在即】在4月20日举办的上海国际车展上，广菲克展出了包括菲翔米兰...,故事与酒缸,-9223357131622972986,2018-01-11 01:31:56,新浪微博,1515597336,故事与酒缸,http://weibo.com/5759812287/FDLHA0DtI
3,5a826ea13ed17ebe58f25c64,Caroline_Nine,消防车🚒//@APP菌:哈哈哈哈哈哈哈哈谈个恋爱还不够生气的//@一个阿呆仔:学猪叫那个真笑...,Caroline_Nine,-9223330358968196918,2018-01-20 16:36:19,新浪微博,1516422278,Caroline_Nine,http://weibo.com/1875936727/FFdIjvCCL
4,5a826ea13ed17ebe58f25c65,qq34314152,【微电影|无处可逃】2017年10月31日上午，[xkl达达黑暗]杨辰和孙彬两位小伙子相约到...,qq34314152,-9223311817460417728,2018-01-21 07:46:52,新浪微博,1516469637,qq34314152,http://weibo.com/2387005072/FFiSHoepC


In [6]:
df_trData = df_trData[~df_trData['content'].isnull()]

In [7]:
df_content = df_trData[['content']]


In [8]:
def extract_emojis(str):
      return ''.join(c for c in str if c not in emoji.UNICODE_EMOJI)

In [9]:
def strQ2B(ustring):
    """把字符串全角转半角"""
    rstring = ""
    for uchar in ustring:
        inside_code=ord(uchar)
        # 全角字符unicode编码从65281~65374 （十六进制 0xFF01 ~ 0xFF5E）
        # 半角字符unicode编码从33~126 （十六进制 0x21~ 0x7E）
        # 而且除空格外,全角/半角按unicode编码排序在顺序上是对应的
        if inside_code==0x3000:
            # 空格 全角为0x3000 半角为0x0020
            inside_code=0x0020
        elif inside_code == 0x3002:
            # 中文句号
            inside_code=0x2e
        else:
            inside_code-=0xfee0
        if inside_code<0x0020 or inside_code>0x7e:      
            rstring += uchar
        else:
            rstring += chr(inside_code)
    return rstring

In [10]:
#df_content['content1'] = df_content.apply(lambda row: strQ2B(row['content1']), axis = 1)

## Merge to One String

In [11]:
df_list = df_content['content'].tolist()

In [12]:
#合併
df_all_in_one = '\n'.join(df_list)
df_all_in_one = ''.join(df_all_in_one.split())

In [13]:
df_all_in_one[0:500]

'【昌河汽车将推2款新车】与北汽集团整车事业本部副本部长、江西昌河汽车有限责任公司副总经理廖雄辉沟通后获悉：“昌河汽车在上海车展发布了全新福瑞达M50S并公布了首款SUV车型，两款车都将于年内上市。“这天我差点出事哦，我去买烧烤，公交车，我看到个人像鬼，多少路我忘记了，后来跟着个杨洋的什么广告，我心定了//@张若昀:一杯不会止一杯【广菲克产销分离JEEP国产在即】在4月20日举办的上海国际车展上，广菲克展出了包括菲翔米兰典藏版、致悦Cross概念车、致悦Mefisto概念车、菲翔跃享版、致悦豪华运动版和菲亚特500等在内的8款车型。而这也是广菲克销售公司挂牌前，在“家门口”的第一次联合演出。消防车🚒//@APP菌:哈哈哈哈哈哈哈哈谈个恋爱还不够生气的//@一个阿呆仔:学猪叫那个真笑出猪声[允悲]//@梨园西池水:哈哈哈哈哈哈哈哈哈哈//@太皇太后您有喜啦:哈哈哈哈哈哈哈哈哈怎么这么好笑ˊ_>ˋ【微电影|无处可逃】2017年10月31日上午，[xkl达达黑暗]杨辰和孙彬两位小伙子相约到盐城市城市商业中心——万达广场游玩，他们在大门口等待商场开门时……[47]【动画电影集散地原创】の《The'

In [14]:
#繁簡/大小寫
openCC = OpenCC('t2s')
df_all_in_one = openCC.convert(df_all_in_one).lower()

KeyboardInterrupt: 

In [ ]:
df_all_in_one[8000:10000]

In [68]:
sent_pattern = u"[\u0030-\u0039\u4e00-\u9fa5\u3040-\u309f\u30a0-\u30ff\u2000-\u2025\u2027\u206f\u3000-\u3001\u3003-\u3009\u3012-\u303f\uff00\uff02-\uff1f\uff20-\uffef]+[\u3002|\uff1f|\uff01|\u3010|\u3011|\u2026]+"
tag_pattern = u"\u0023+[\w\W\u0020-\u007f\u4e00-\u9fa5\u3040-\u309f\u30a0-\u30ff]+?\u0023"
id_pattern = u"\u0040+[\w\W\u0020-\u007f\u4e00-\u9fa5\u3040-\u309f\u30a0-\u30ff]+?\u003A"
url_pattern = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
url_pattern1 = "//(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
mail_pattern = "^\\s*\\w+(?:\\.{0,1}[\\w-]+)*@[a-zA-Z0-9]+(?:[-.][a-zA-Z0-9]+)*\\.[a-zA-Z]+\\s*$"
other_emoji = u"\[+[\u0020-\u007f\u4e00-\u9fa5\u3040-\u309f\u30a0-\u30ff]+?\]"

In [ ]:
txt = "@APP菌:哈哈哈哈哈哈哈哈谈个恋爱还不够生气的//@一个阿呆仔:学猪叫那个真笑出猪声[允悲]//@梨园西池水:哈哈哈哈哈哈哈哈哈哈//@太皇太后您有喜啦:哈哈哈哈哈哈哈哈哈怎么这么好笑ˊ_>ˋ【微电影|无处可逃】2017年10月31日上午，[xkl达达黑暗]杨辰和孙彬两位小伙子相约到盐城市城市商业中心——万达广场游玩，他们在大门口等待商场开门时……[47]【动画电影集散地原创】の《The"
re.findall(sent_pattern, txt)

In [90]:
pa = tag_pattern+'|'+id_pattern+'|'+url_pattern+'|'+mail_pattern+'|'+other_emoji+'|'+url_pattern1

In [91]:
#忽略特定格式
handle = re.sub(pa,'',df_all_in_one)

In [92]:
handle_wo_emoji = extract_emojis(handle)

In [102]:
handle_wo_emoji[0:1000]

'【昌河汽车将推2款新车】与北汽集团整车事业本部副本部长、江西昌河汽车有限责任公司副总经理廖雄辉沟通后获悉：“昌河汽车在上海车展发布了全新福瑞达m50s并公布了首款suv车型，两款车都将于年内上市。“ 这天我差点出事哦，我去买烧烤，公交车，我看到个人像鬼，多少路我忘记了，后来跟着个杨洋的什么广告，我心定了张若昀: 一杯不会止一杯 【广菲克产销分离jeep国产在即】在4月20日举办的上海国际车展上，广菲克展出了包括菲翔米兰典藏版、致悦cross概念车、致悦mefisto概念车、菲翔跃享版、致悦豪华运动版和菲亚特500等在内的8款车型。而这也是广菲克销售公司挂牌前，在“家门口”的第一次联合演出。 消防车菌:哈哈哈哈哈哈哈哈谈个恋爱还不够生气的一个阿呆仔:学猪叫那个真笑出猪声梨园西池水: 哈哈哈哈哈哈哈哈哈哈太皇太后您有喜啦:哈哈哈哈哈哈哈哈哈怎么这么好笑ˊ_>ˋ 【微电影|无处可逃】2017年10月31日上午，杨辰和孙彬两位小伙子相约到盐城市城市商业中心——万达广场游玩，他们在大门口等待商场开门时…… 【动画电影集散地原创】の《the polar express》〔20050121〕某天我们突然对自己曾经 【bee dragon打造完美vip丰田皇冠】bee dragon打造的标志性vip风格。bee dragon 定制前保险杠，前大灯调色搭配hid 雾灯，霸气的前脸，不怒自威。vip改装车的效果总是令人惊叹。专业定制的气动悬挂，趴地下是必须的。外倾角并没有夸张到接受不了，这么一辆车停在路边，没有谁不会去多看两眼把。 【mini paceman 皮卡】mini发布了paceman adventure concept概念车的官方图片。这款车仅会生产一台，目前mini并没有计划将其量产。paceman adventure concept概念车是基于paceman普通版车型改进而来的一款皮卡车型，该车采用绿色涂装，车顶配有探照灯以及置物架。 人社部财政部近日联合印发了《企业年金办法》，自2018年2月1日起施行。 企业缴费每年不超过工资总额的8%，企业和职工个人缴费合计不超过12%。具体由企业和职工协商确定。企业年金基金实行完全积累，为每个职工建立个人账户，按照国家有关规定投资运营；收益并入企业年金基金  【科鲁兹两厢版】外观设计上，科鲁兹两厢版除了尾部设计外均与三厢版一致。

In [74]:
#title_pattern = u"\u3010+[\w\W\u0020-\u007f\u4e00-\u9fa5\u3040-\u309f\u30a0-\u30ff]+?\u3011"
#title_list = re.findall(title_pattern,handle_wo_emoji)
#title_list = [re.sub('【|】', '', title) for title in title_list ]
#df_title = pandas.DataFrame(title_list)
#df_title.to_csv('df_title.csv', sep='\t', encoding='utf-8')
#handle_wo_title = re.sub(title_pattern, '', handle_wo_emoji)

In [103]:
split_row = re.split('。+|！+|？+|…+|/+|】|：|【', handle_wo_emoji)

In [104]:
tmp = pandas.DataFrame(split_row)

In [105]:
tmp.head()

,0
0,
1,昌河汽车将推2款新车
2,与北汽集团整车事业本部副本部长、江西昌河汽车有限责任公司副总经理廖雄辉沟通后获悉
3,“昌河汽车在上海车展发布了全新福瑞达m50s并公布了首款suv车型，两款车都将于年内上市
4,“ 这天我差点出事哦，我去买烧烤，公交车，我看到个人像鬼，多少路我忘记了，后来跟着个杨洋的什...


In [106]:
tmp.count()

0    1454250
dtype: int64

In [107]:
tmp = tmp.rename({0: "sent"}, axis='columns')

In [108]:
tmp = tmp[tmp['sent'].apply(lambda row:len(row)>=2)]

In [111]:
tmp[0:1000]

,sent
1,昌河汽车将推2款新车
2,与北汽集团整车事业本部副本部长、江西昌河汽车有限责任公司副总经理廖雄辉沟通后获悉
3,“昌河汽车在上海车展发布了全新福瑞达m50s并公布了首款suv车型，两款车都将于年内上市
4,“ 这天我差点出事哦，我去买烧烤，公交车，我看到个人像鬼，多少路我忘记了，后来跟着个杨洋的什...
5,广菲克产销分离jeep国产在即
6,在4月20日举办的上海国际车展上，广菲克展出了包括菲翔米兰典藏版、致悦cross概念车、致悦...
7,而这也是广菲克销售公司挂牌前，在“家门口”的第一次联合演出
8,消防车菌:哈哈哈哈哈哈哈哈谈个恋爱还不够生气的一个阿呆仔:学猪叫那个真笑出猪声梨园西池水:...
9,微电影|无处可逃
10,2017年10月31日上午，杨辰和孙彬两位小伙子相约到盐城市城市商业中心——万达广场游玩，他...


In [112]:
tmp.to_csv('tmp1.csv', sep='\t', encoding='utf-8')

## TEST: Text Filtering

* http: or https: 
* 【】subtitle
* #tag# @person_
* emoji

In [20]:
import re

In [40]:
zh_char = u"[\u4e00-\u9fa5\u3040-\u309f\u30a0-\u30ff]+"
zh_punc = u"[\u2000-\u206f\u3000-\u303f\uff00-\uffef]+"
zh_patt = u"[\u0030-\u0039\u4e00-\u9fa5\u2000-\u206f\u3000-\u303f\u3040-\u309f\u30a0-\u30ff\uff00-\uffef]+"
zh_half_punc = u"[\u2000-\u2025\u2027\u206f\u3000-\u3001\u3003-\u3009\u3012-\u303f\uff00\uff02-\uff1f\uff20-\uffef]+"
en_char = u"[\u0020-\u007f]+"
sent_pattern = u"[\u0030-\u0039\u4e00-\u9fa5\u3040-\u309f\u30a0-\u30ff\u2000-\u2025\u2027\u206f\u3000-\u3001\u3003-\u3009\u3012-\u303f\uff00\uff02-\uff1f\uff20-\uffef]+[\u3002|\uff1f|\uff01|\u3010|\u3011|\u2026]+"

In [109]:
df_all_in_one[0:5000]

'【昌河汽车将推2款新车】与北汽集团整车事业本部副本部长、江西昌河汽车有限责任公司副总经理廖雄辉沟通后获悉：“昌河汽车在上海车展发布了全新福瑞达M50S并公布了首款SUV车型，两款车都将于年内上市。“ 这天我差点出事哦，我去买烧烤，公交车，我看到个人像鬼，多少路我忘记了，后来跟着个杨洋的什么广告，我心定了//@张若昀: 一杯不会止一杯 【广菲克产销分离JEEP国产在即】在4月20日举办的上海国际车展上，广菲克展出了包括菲翔米兰典藏版、致悦Cross概念车、致悦Mefisto概念车、菲翔跃享版、致悦豪华运动版和菲亚特500等在内的8款车型。而这也是广菲克销售公司挂牌前，在“家门口”的第一次联合演出。 消防车🚒//@APP菌:哈哈哈哈哈哈哈哈谈个恋爱还不够生气的//@一个阿呆仔:学猪叫那个真笑出猪声[允悲]//@梨园西池水: 哈哈哈哈哈哈哈哈哈哈//@太皇太后您有喜啦:哈哈哈哈哈哈哈哈哈怎么这么好笑ˊ_>ˋ 【微电影|无处可逃】2017年10月31日上午，[xkl达达黑暗]杨辰和孙彬两位小伙子相约到盐城市城市商业中心——万达广场游玩，他们在大门口等待商场开门时…… [47]【动画电影集散地原创】の《The Polar Express》〔20050121〕某天我们突然对自己曾经 【Bee Dragon打造完美VIP丰田皇冠】Bee Dragon打造的标志性VIP风格。Bee Dragon 定制前保险杠，前大灯调色搭配HID 雾灯，霸气的前脸，不怒自威。VIP改装车的效果总是令人惊叹。专业定制的气动悬挂，趴地下是必须的。外倾角并没有夸张到接受不了，这么一辆车停在路边，没有谁不会去多看两眼把。 #我有红唇给我红包#【MINI PACEMAN 皮卡】MINI发布了PACEMAN Adventure Concept概念车的官方图片。这款车仅会生产一台，目前MINI并没有计划将其量产。PACEMAN Adventure Concept概念车是基于PACEMAN普通版车型改进而来的一款皮卡车型，该车采用绿色涂装，车顶配有探照灯以及置物架。 人社部财政部近日联合印发了《企业年金办法》，自2018年2月1日起施行。 企业缴费每年不超过工资总额的8%，企业和职工个人缴费合计不超过12%。具体由企业和职工协商确定。企业年金基金实行完全积累，为每个职工建立个人账户，按照国家有关规定投资运营；收

In [116]:
content = df_all_in_one[0:5000]
content = ' '.join(content.split())
content = ''.join(c for c in content if c not in emoji.UNICODE_EMOJI)
content = re.sub(pa, '', content)
content

'【昌河汽车将推2款新车】与北汽集团整车事业本部副本部长、江西昌河汽车有限责任公司副总经理廖雄辉沟通后获悉：“昌河汽车在上海车展发布了全新福瑞达M50S并公布了首款SUV车型，两款车都将于年内上市。“ 这天我差点出事哦，我去买烧烤，公交车，我看到个人像鬼，多少路我忘记了，后来跟着个杨洋的什么广告，我心定了// 一杯不会止一杯 【广菲克产销分离JEEP国产在即】在4月20日举办的上海国际车展上，广菲克展出了包括菲翔米兰典藏版、致悦Cross概念车、致悦Mefisto概念车、菲翔跃享版、致悦豪华运动版和菲亚特500等在内的8款车型。而这也是广菲克销售公司挂牌前，在“家门口”的第一次联合演出。 消防车//哈哈哈哈哈哈哈哈谈个恋爱还不够生气的//学猪叫那个真笑出猪声[允悲]// 哈哈哈哈哈哈哈哈哈哈//哈哈哈哈哈哈哈哈哈怎么这么好笑ˊ_>ˋ 【微电影|无处可逃】2017年10月31日上午，[xkl达达黑暗]杨辰和孙彬两位小伙子相约到盐城市城市商业中心——万达广场游玩，他们在大门口等待商场开门时…… [47]【动画电影集散地原创】の《The Polar Express》〔20050121〕某天我们突然对自己曾经 【Bee Dragon打造完美VIP丰田皇冠】Bee Dragon打造的标志性VIP风格。Bee Dragon 定制前保险杠，前大灯调色搭配HID 雾灯，霸气的前脸，不怒自威。VIP改装车的效果总是令人惊叹。专业定制的气动悬挂，趴地下是必须的。外倾角并没有夸张到接受不了，这么一辆车停在路边，没有谁不会去多看两眼把。 【MINI PACEMAN 皮卡】MINI发布了PACEMAN Adventure Concept概念车的官方图片。这款车仅会生产一台，目前MINI并没有计划将其量产。PACEMAN Adventure Concept概念车是基于PACEMAN普通版车型改进而来的一款皮卡车型，该车采用绿色涂装，车顶配有探照灯以及置物架。 人社部财政部近日联合印发了《企业年金办法》，自2018年2月1日起施行。 企业缴费每年不超过工资总额的8%，企业和职工个人缴费合计不超过12%。具体由企业和职工协商确定。企业年金基金实行完全积累，为每个职工建立个人账户，按照国家有关规定投资运营；收益并入企业年金基金  【科鲁兹两厢版】外观设计上，科鲁兹两厢版除了尾部设计外均与三厢版一致。

In [59]:
re.split('。|！|？|…|//|： ',df_all_in_one[0:5000])

['【昌河汽车将推2款新车】与北汽集团整车事业本部副本部长、江西昌河汽车有限责任公司副总经理廖雄辉沟通后获悉：“昌河汽车在上海车展发布了全新福瑞达M50S并公布了首款SUV车型，两款车都将于年内上市',
 '“ 这天我差点出事哦，我去买烧烤，公交车，我看到个人像鬼，多少路我忘记了，后来跟着个杨洋的什么广告，我心定了',
 '@张若昀: 一杯不会止一杯 【广菲克产销分离JEEP国产在即】在4月20日举办的上海国际车展上，广菲克展出了包括菲翔米兰典藏版、致悦Cross概念车、致悦Mefisto概念车、菲翔跃享版、致悦豪华运动版和菲亚特500等在内的8款车型',
 '而这也是广菲克销售公司挂牌前，在“家门口”的第一次联合演出',
 ' 消防车🚒',
 '@APP菌:哈哈哈哈哈哈哈哈谈个恋爱还不够生气的',
 '@一个阿呆仔:学猪叫那个真笑出猪声[允悲]',
 '@梨园西池水: 哈哈哈哈哈哈哈哈哈哈',
 '@太皇太后您有喜啦:哈哈哈哈哈哈哈哈哈怎么这么好笑ˊ_>ˋ 【微电影|无处可逃】2017年10月31日上午，[xkl达达黑暗]杨辰和孙彬两位小伙子相约到盐城市城市商业中心——万达广场游玩，他们在大门口等待商场开门时',
 '',
 ' [47]【动画电影集散地原创】の《The Polar Express》〔20050121〕某天我们突然对自己曾经 【Bee Dragon打造完美VIP丰田皇冠】Bee Dragon打造的标志性VIP风格',
 'Bee Dragon 定制前保险杠，前大灯调色搭配HID 雾灯，霸气的前脸，不怒自威',
 'VIP改装车的效果总是令人惊叹',
 '专业定制的气动悬挂，趴地下是必须的',
 '外倾角并没有夸张到接受不了，这么一辆车停在路边，没有谁不会去多看两眼把',
 ' #我有红唇给我红包#【MINI PACEMAN 皮卡】MINI发布了PACEMAN Adventure Concept概念车的官方图片',
 '这款车仅会生产一台，目前MINI并没有计划将其量产',
 'PACEMAN Adventure Concept概念车是基于PACEMAN普通版车型改进而来的一款皮卡车型，该车采用绿色涂装，车顶配有探照灯以及置物架',
 ' 人社部财政部近日联合印发了《企业年金办法》，自2018年2月1日起施行',
 ' 企业缴费每年不超过工资总额的8

In [17]:
from nltk.tokenize import TweetTokenizer 
tw = TweetTokenizer()
#tw.tokenize(df_all_in_one[5000:6000])


In [23]:
pa = u"\u0023+[\u4e00-\u9fa5\u3040-\u309f\u30a0-\u30ff]+\u0023"

In [29]:
b = re.findall(pa,df_all_in_one)

In [30]:
b

['#我有红唇给我红包#',
 '#和工行一起云跨年#',
 '#法治微课堂#',
 '#琅琊榜之风起长林#',
 '#戴佩妮#',
 '#让红包飞#',
 '#周口新鲜事#',
 '#我有红唇给我红包#',
 '#中国人寿#',
 '#平安守护#',
 '#快乐冰雪季#',
 '#陈伟霆#',
 '#轻松一刻#',
 '#公司研报#',
 '#属于我的金球时刻#',
 '#林峯#',
 '#林峯大泼猴#',
 '#平安守护#',
 '#张艺兴#',
 '#张艺兴代言浦发信用卡#',
 '#张艺兴梦想起飞#',
 '#张艺兴湖南卫视跨年#',
 '#张艺兴偶像练习生#',
 '#张艺兴黄金瞳#',
 '#媒体看中行#',
 '#我有红唇给我红包#',
 '#重装突击#',
 '#临沂房价#',
 '#尊重也是一种温暖#',
 '#时空偷渡少女#',
 '#绿地黄浦滨江豪宅甲醛超标#',
 '#最美国风花旦#',
 '#宝宝的少女心#',
 '#庄粉需知#',
 '#万达乐园欢乐无限#',
 '#搜狐新闻资讯版#',
 '#民生峰享信用卡#',
 '#王源大主宰#',
 '#电视剧大主宰#',
 '#王源#',
 '#餐饮年会发豪车#',
 '#王俊凯#',
 '#贵州看看#',
 '#贵州突发#',
 '#小狐狸偷鱼#',
 '#寻梦环游记#',
 '#吐槽大会#',
 '#南宁业之峰装饰#',
 '#家居创意#',
 '#公安厅大院#',
 '#交警红绿灯#',
 '#冬季大雾天气交通安全提示#',
 '#新南京唐僧肉#',
 '#微博股票#',
 '#财经视频#',
 '#微博财经#',
 '#西宁招聘#',
 '#请丝芭公司公平对待莫寒#',
 '#爱奇艺体育澳网奇妙之旅#',
 '#大连生活#',
 '#王俊凯#',
 '#寒门博士之死#',
 '#警民连心#',
 '#瓜子二手车疑似资金链断裂#',
 '#春节#',
 '#外汇黄金分析师#',
 '#汇添富基金携手城市之光#',
 '#整点路况#',
 '#马蓉#',
 '#宋喆#',
 '##鹿晗#',
 '#秒拍#',
 '#武汉同城#',
 '#武汉楼市#',
 '#说到街舞就能想到易烊千玺#',
 '#百日平安会战#',
 '#早票网票据交易#',
 '#西宁房产信息#',
 '#安亭瑞仕锦庭#'

In [ ]:
df_content_str = df_content.apply(lambda row: filterStr(row), axis = 1)

## 分句

## 左右熵